# TimerThread

## `@task` Decorator

In [1]:
import functools


class task(object):

    def __init__(self, trigger, interval):
        self.trigger = trigger
        self.interval = interval

    def __call__(self, fn):
        @functools.wraps(fn)
        def sched(*args, **kwargs):
            scheduler_kwargs = {
                'trigger': self.trigger,
                'interval': self.interval,
                'fn': fn,
                'args': args,
                'kwargs': kwargs
            }            
            return Scheduler.create(**scheduler_kwargs)
        fn.sched = sched
        return fn

## Scheduler

Based on [Lib/threading.py](https://github.com/python/cpython/blob/3.7/Lib/threading.py#L1153)

In [2]:
TIMEFORMAT = '%Y-%m-%d %H:%M:%S %Z'

import threading
import time


class Scheduler(threading.Thread):
    
    def __init__(self, trigger, interval, fn, args=(), kwargs={}):
        super().__init__()        
        if trigger not in ['delay', 'recur']:
            raise ValueError('trigger must be "delay" or "recur"')             
        self.trigger = trigger
        self.interval = interval
        self.fn = fn
        self.args = args
        self.kwargs = kwargs        
        self._event = threading.Event()
        self._stopped = True
        self._result = []
    
    # Scheduler constructor
    @classmethod
    def create(cls, **scheduler_kwargs):
        return cls(**scheduler_kwargs)
    
    @property   
    def stopped(self):
        return self._stopped
    
    @property
    def result(self):
        return dict(self._result)
        
    def run(self):
        self._stopped = self._event.is_set()  # False
        if self.trigger == 'delay':
            self._delay()
        elif self.trigger == 'recur':
            self._recur()           
            
    def _delay(self):
        if not self._stopped:
            self._event.wait(timeout=self.interval)
            self._execute()
            
    def _recur(self):
        while not self._stopped:
            self._execute()
            self._event.wait(timeout=self.interval)
                                    
    def _execute(self):
        self._fn(self.args, self.kwargs)
        
    def _fn(self, args, kwargs):
        return_value = self.fn(*args, **kwargs)
        if return_value:
            timestamp = time.strftime(TIMEFORMAT, time.localtime())
            self._result.append( (timestamp, return_value) )
    
    # threads can only be started once
    # i.e. cannot start again once cancelled
    def cancel(self):
        self._event.set()
        self._stopped = self._event.is_set()  # True

## Install TimerThread

In [3]:
!pip install timerthread

from timerthread import task, Scheduler

## Example

In [4]:
import time

@task('recur', 3)
def now(cost=0):
    time.sleep(cost)
    print( time.strftime('%Y-%m-%d %H:%M:%S %Z', time.localtime()) ) 

In [5]:
timer = now.sched(cost=1)
timer.start()

In [6]:
timer.cancel()

2020-12-13 04:28:49 CET
